In [18]:
from bs4 import BeautifulSoup
import requests
import os
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument


def get_documents(file_extension, url, time_indicator, directory):
    r  = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data)
    
    url_list = []
    
    for link in soup.find_all('a'):
        try:
            if file_extension in link.get('href'):
                url_list.append(link.get('href'))
        except:
            None
            
    os.chdir(directory)
    
    for index, url in enumerate(url_list):
        try:
            r = requests.get(url)

            write_path = time_indicator + "_file_" + str(index) + file_extension

            with open(write_path,'wb') as f: 
                f.write(r.content) 

            with open(write_path, 'rb') as fp:
                parser = PDFParser(fp)
                doc = PDFDocument(parser)

                try:
                    subject = doc.info[0]['Subject'].decode("utf-8") 
                    subject = subject.replace('/', '')
                except:
                    subject = "no_subject" + str(index)

                try:
                    creation = doc.info[0]['CreationDate'].decode("utf-8")[2:-4]
                except:
                    creation = "no_creation" + str(index)

            new_path = creation + '_' + subject + file_extension

            os.rename(write_path, new_path)
        except FileExistsError:
            print("duplicate")
            os.remove(write_path)
            pass

    os.chdir("..")

In [24]:
for year in range(2015, 2020):
    url = "https://ir.aboutamazon.com/sec-filings/" + str(year) + "/default.aspx"
    
    get_documents(file_extension = '.pdf', 
                  url = url, 
                  time_indicator = str(year),
                  directory = "Amazon")

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'Walmart'